In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
import os
from itertools import product
from functions import (load_parameters, load_generation_data, load_price_data, generate_randomized_generation,
generate_rt_scenarios, plot_generation_data, plot_randomized_generation, plot_scenarios_for_generator, plot_rt_scenarios)

generation_data, I, T = load_generation_data(date_filter="2022-07-18")
S, R, P_RT, K, K0, M1, M2 = load_parameters(I, T, generation_data)
P_DA, P_PN = load_price_data()

R_ind = R[I-1:I, :]  # shape: (1, T)
R_agg = R[:I-1, :]  # shape: (I-1, T)

# --------------------------------------------------------------------------------------------------

agg = gp.Model("agg")
agg.Params.MIPGap = 0.00001

alpha = agg.addVars(T, vtype=GRB.CONTINUOUS, lb=0, name="alpha")
beta_plus = agg.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="beta_plus")
beta_minus = agg.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="beta_minus")
M = max(sum(R_agg[i, t, s] for i in range(I-1)) for t in range(T) for s in range(S))
z = agg.addVars(T, S, vtype=GRB.BINARY, name="z")
prob = np.full(S, 1 / S)

agg.update()

obj = gp.quicksum(P_DA[t] * alpha[t] for t in range(T)) + gp.quicksum(
    prob[s] * (P_RT[t, s] * beta_plus[t, s] - P_PN[t] * beta_minus[t, s])
    for t in range(T)
    for s in range(S)
)

agg.setObjective(obj, GRB.MAXIMIZE)

for t, s in product(range(T), range(S)):
    agg.addConstr(gp.quicksum(R_agg[i, t, s] for i in range(I-1)) - alpha[t] == beta_plus[t, s] - beta_minus[t, s])
    agg.addConstr(gp.quicksum(R_agg[i, t, s] for i in range(I-1)) >= beta_plus[t, s])
    agg.addConstr(beta_plus[t, s] <= M * z[t, s])
    agg.addConstr(beta_minus[t, s] <= M * (1 - z[t, s]))

agg.optimize()

if agg.status == GRB.OPTIMAL:
    print("Optimal solution found!")
    print(f"Objective value: {agg.objVal}")
else:
    print("No optimal solution found.")
    
R_agg_vals = np.sum(R_agg, axis=0)
alpha_vals = np.tile(np.array([alpha[t].X for t in range(T)])[:, None], (1, S))
beta_plus_vals = np.array([[beta_plus[t, s].X for s in range(S)] for t in range(T)])
beta_minus_vals = np.array([[beta_minus[t, s].X for s in range(S)] for t in range(T)])

✅ 총 5개 파일을 불러왔습니다: 1201.csv, 137.csv, 401.csv, 524.csv, 89.csv
📊 데이터 Shape: I=5, T=24, S=30
✅ 시뮬레이션 초기화 완료: S=30, Randomness='high', M1=722.00, M2=1957.00
Set parameter MIPGap to value 1e-05
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D81)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
MIPGap  1e-05

Optimize a model with 2880 rows, 2184 columns and 5760 nonzeros
Model fingerprint: 0xb3840f1c
Variable types: 1464 continuous, 720 integer (720 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [2e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective 1076609.7550
Presolve removed 2790 rows and 2093 columns
Presolve time: 0.02s
Presolved: 90 rows, 91 columns, 210 nonzeros
Found heuristic solution: objective 1401117.9879
Variable types: 61 continuous, 30 integer (30 binary)

Root relaxati

### 모델 DER only with Contribution

In [ ]:
only = gp.Model("only")

i = I-1

x = only.addVars(T, vtype=GRB.CONTINUOUS, lb=0, name="x")
delta = only.addVars(T, vtype=GRB.CONTINUOUS, lb=0, name="delta")
y_plus = only.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_plus")
y_minus = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_minus")
Ma = int(R_ind[0, :, :].max())
Mb = int((beta_plus_vals + R_ind[0]).max())
Mc = Mb
a = only.addVars(T, S, vtype=GRB.BINARY, name="a")
b = only.addVars(T, S, vtype=GRB.BINARY, name="b")
c = only.addVars(T, S, vtype=GRB.BINARY, name="c")
delta_plus = only.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="delta_plus")     # Δᵟ⁺
delta_minus = only.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="delta_minus")   # Δᵟ⁻
delta_plus_baseline = only.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="delta_plus_baseline")   # Δ⁺
delta_minus_baseline = only.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="delta_minus_baseline") # Δ⁻

only.update()

obj = gp.quicksum(
    P_DA[t] * x[i, t] for i in range(I) for t in range(T)
) + gp.quicksum(
    1/S * (P_RT[t, s] * y_plus[i, t, s] - P_PN[t] * y_minus[i, t, s])
    for i in range(I)
    for t in range(T)
    for s in range(S)
)

only.setObjective(obj, GRB.MAXIMIZE)

for t, s, in product(range(T), range(S)):
    x

only.optimize()

if only.status == GRB.OPTIMAL:
    print("Optimal solution found!")
    print(f"Objective value: {only.objVal}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D81)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7200 rows, 10920 columns and 14400 nonzeros
Model fingerprint: 0x006025b5
Variable types: 7320 continuous, 3600 integer (3600 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Found heuristic solution: objective 1314732.4595
Presolve removed 7200 rows and 10920 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.01 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 1.54373e+06 1.31473e+06 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.543726381360e+06, best bound 1.543726381360e+06, gap 0.0000%
Optimal solution found!
Objective value: 1543726.3813601548

### 결과 분석

#### 수익 분석

In [16]:
# Day-ahead 수익 비교
total_da_profit_obj = 0
for i in range(I):
    for t in range(T):
        total_da_profit_obj += P_DA[t] * x[i,t].x

# Real-time 수익 비교
total_rt_profit_obj = 0
for i in range(I):
    for t in range(T):
        for s in range(S):
            rt_profit_obj = P_RT[t, s] * y_plus[i, t, s].x
            total_rt_profit_obj += 1/S * rt_profit_obj

# 패널티 비용 비교
total_penalty_cost_obj = 0
for i in range(I):
    for t in range(T):
        for s in range(S):
            penalty_cost_obj = P_PN[t] * y_minus[i, t, s].x
            total_penalty_cost_obj += 1/S * penalty_cost_obj

# 총 시스템 이익 (목적 함수 기반)
total_system_profit_obj = total_da_profit_obj + total_rt_profit_obj - total_penalty_cost_obj

# 결과 출력
print(f"DA: {total_da_profit_obj:.2f}")
print(f"RT: {total_rt_profit_obj:.2f}")
print(f"Penalty: {total_penalty_cost_obj:.2f}")
print(f"목적 함수 기반 총 이익 (_obj): {total_system_profit_obj:.2f}")

DA: 339706.34
RT: 1253062.97
Penalty: 49042.93
목적 함수 기반 총 이익 (_obj): 1543726.38


#### 하루 개인별 커밋량 합 분석

In [17]:
sum_x = 0
for t in range(T):
    time_sum = sum(x[i,t].x for i in range(I))
    print(f"시간 {t}: {time_sum}")
    sum_x += time_sum
print(f"총 하루 commitment: {sum_x:.2f}")


시간 0: 0.0
시간 1: 0.0
시간 2: 0.0
시간 3: 0.0
시간 4: 0.0
시간 5: 0.0
시간 6: 2.0
시간 7: 6.0
시간 8: 23.0
시간 9: 114.0
시간 10: 161.0
시간 11: 0.0
시간 12: 0.0
시간 13: 0.0
시간 14: 1041.0
시간 15: 0.0
시간 16: 542.0
시간 17: 0.0
시간 18: 0.0
시간 19: 238.0
시간 20: 70.0
시간 21: 10.0
시간 22: 0.0
시간 23: 0.0
총 하루 commitment: 2207.00


#### 개인 수익 분석

In [18]:
# 모든 der의 profit 합계 계산
total_der_profit = 0
der_profit = {}
for i in range(I):
    # 각 der i의 profit 합계 계산
    der_profit[i] = sum(P_DA[t] * x[i,t].x + sum(1/S * (P_RT[t,s] * y_plus[i,t,s].x - P_PN[t] * y_minus[i,t,s].x) for s in range(S)) for t in range(T))
    total_der_profit += der_profit[i]

print("\nder_profit:")
for i in range(I):
    print(f"[{i}] {der_profit[i]:.2f}")

print(f"\n모든 der의 profit 합계: {total_der_profit:.2f}")

der_hourly_profit = np.zeros((I, T, 4))

for i in range(I):
    for t in range(T):
        da_profit = P_DA[t] * x[i, t].x
        rt_profit = sum(1 / S * (P_RT[t, s] * y_plus[i, t, s].x) for s in range(S))
        pen_cost = sum(1 / S * (P_PN[t] * y_minus[i, t, s].x) for s in range(S))
        hourly_total = da_profit + rt_profit - pen_cost

        der_hourly_profit[i, t, 0] = da_profit  # DA profit
        der_hourly_profit[i, t, 1] = rt_profit  # RT profit
        der_hourly_profit[i, t, 2] = pen_cost   # Penalty cost
        der_hourly_profit[i, t, 3] = hourly_total



der_profit:
[0] 201827.57
[1] 331310.50
[2] 406246.12
[3] 442248.26
[4] 162093.93

모든 der의 profit 합계: 1543726.38


### 결과 저장

In [19]:
# R_sum = {(i, t): sum(R[i, t, s] for s in range(S)) / S for i in range(I) for t in range(T)}
# R_df = pd.DataFrame({i: {t: R_sum[i, t] for t in range(T)} for i in range(I)})
# R_df.to_csv('result_R.csv', index=False)

# pd.DataFrame(list(P_DA.items()), columns=['time', 'price']).to_csv('result_P_DA.csv', index=False)

# total_x_only = {t: sum(x[i,t].x for i in range(I)) for t in range(T)}
# total_x_only_df = pd.DataFrame({'total_x_only': total_x_only})
# total_x_only_df.to_csv('result/result_base_totalX.csv', index=False)

only_obj = pd.DataFrame({'only_obj': [only.objVal]})
only_obj.to_csv('result/result_only_obj.csv', index=False)

only_profit_value = pd.DataFrame({'only_profit_value': der_profit})
only_profit_value.to_csv('result/result_only_profit.csv', index=False)

data_list = []

for i in range(I):
    for t in range(T):
        data_list.append([i, t, 
                          der_hourly_profit[i, t, 0],  # DA profit
                          der_hourly_profit[i, t, 1],  # RT profit
                          der_hourly_profit[i, t, 2],  # Penalty cost
                          der_hourly_profit[i, t, 3]]) # Hourly total profit

# Pandas DataFrame 생성
df = pd.DataFrame(data_list, columns=['DER', 'Hour', 'hourly_da', 'hourly_rt', 'hourly_pen', 'hourly_total'])

# CSV 파일로 저장
df.to_csv('result/result_only_hourly_profit.csv', index=False)
